<div class="alert alert-block alert-success">
<b>This simulation runs a circuit with a varying number of gates for the purpose of detecting depolarization error. </b>
<div>Obs. 1: The data is saved in external files so that it can be worked on without incurring new costs.</div>
<div>Obs. 2: For new simulation parameters, save in a file with a different name.</div>
</div>

# Instructions if you are running outside AWS cloud

1. Install aws-cli:

`sudo apt install aws-cli`

2. Setup aws credentials:

`aws configure --profile "default"`

3. Create virtual environment 

`python -m venv /path/to/new/virtual/environment`

4. Activate virtual environment

`source /path/to/new/virtual/environment/bin/activate`

### Instalando e Atualizando Dependencias

In [ ]:
# Install this modules on your virtual environment: an open source library for interacting
# with quantum computing devices and simulators on Amazon Braket
!pip install -r requirements.txt

In [32]:
# Para atualizar as dependencias, caso tenha sido incluidas novas bibliotecas no projeto
!pip freeze > requirements.txt

### Importando classes de inicialização de configuração, devices e pricing

In [9]:
%run devices/aws_backends.ipynb

Dispositivos disponíveis na AWS Braket:
0: Nome: Ankaa-3, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
1: Nome: Aquila, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/quera/Aquila
2: Nome: Aria 1, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1
3: Nome: Aria 2, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2
4: Nome: Forte 1, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1
5: Nome: Garnet, Tipo: AwsDeviceType.QPU, ARN: arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet
6: Nome: SV1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/sv1
7: Nome: TN1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/tn1
8: Nome: dm1, Tipo: AwsDeviceType.SIMULATOR, ARN: arn:aws:braket:::device/quantum-simulator/amazon/dm1
Backends ativos: ['ANKAA_3', 'AQUILA', 'ARIA 1', 'FORTE 1', 'GARNET

In [18]:
%run config/load_configuration.ipynb

In [11]:
%run pricing/Pricing.ipynb


📋 Tabela de Preços das QPUs:
Família de QPU | Preço por Tarefa (USD) | Preço por Shot (USD)
------------------------------------------------------------
Forte      | $0.30000              | $0.08000             
Aria       | $0.30000              | $0.03000             
Garnet     | $0.30000              | $0.00145             
Aquila     | $0.30000              | $0.01000             
Ankaa      | $0.30000              | $0.00090             
------------------------------------------------------------


In [19]:
%run execution/Execution.ipynb

### Criando o Circuito Quântico

In [20]:
from braket.circuits import Circuit

def create_circuit():
    """Cria um circuito de Bell e retorna o objeto Circuit."""
    circuit = Circuit().h(0).cnot(0, 1).measure(0).measure(1)
    return circuit

### Escolhendo o Backend 

In [21]:
# Listar backends ativos
print("Backends ativos:", list(Backend.__members__.keys()))

Backends ativos: ['ANKAA_3', 'AQUILA', 'ARIA 1', 'FORTE 1', 'GARNET', 'SV1', 'TN1', 'DM1']


### Definição dos parâmetros perfil, backend e circuito 

In [15]:
# Escolha o perfil desejado
perfil = ConfigProfile.EXECUTION  # Pode ser: ConfigProfile.CONECTIVIDADE, ConfigProfile.TESTE, ConfigProfile.EXECUTION

# Escolher um backend específico
backend = Backend.GARNET  # Por exemplo, selecionar o simulador SV1

# Criar um circuito
circuit = create_circuit()

### Testando o código de forma síncrona (fila livre)

In [16]:
# Executar o circuito com o perfil e backend selecionados
run_circuit_sync(perfil, backend, circuit)

⚠️ Custo estimado: $1.750000


Deseja continuar? (s/n):  n


❌ Execução cancelada pelo usuário.


### Testando o código de forma assíncrona (fila ocupada)

In [31]:
# Executar o circuito com o perfil e backend selecionados mas de forma assincrona
task_arn = run_circuit_async(perfil, backend, circuit)
print("Task ARN recebida:", task_arn)

Executando (ASSÍNCRONO) com 1 shots no perfil 'perfil_1-conectividade' no backend 'GARNET'
📌 Task ARN: arn:aws:braket:eu-north-1:709381776175:quantum-task/89154895-e214-4b08-b706-ad28acac15d2
Task ARN recebida: arn:aws:braket:eu-north-1:709381776175:quantum-task/89154895-e214-4b08-b706-ad28acac15d2
